### 实施 FunkSVD

在此 notebook 中，我们将自己编写实施 FunkSVD 的函数，它会遵守你在上个视频中看到的步骤。如果你认为你还没有准备好自己完成这个任务，可以跳到下个视频，观看我是如何完成这些步骤的。

我们将用之前使用的数据子集检测我们的算法。首先请运行以下单元格。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


`1.` 你将使用 **user_movie_subset** 矩阵演示你的 FunkSVD 算法将收敛。在以下单元格中，请参考注释和文档字符串编写你自己的 FunkSVD 函数。你也可以自己完成这些函数，不参考注释部分。你可以随意增减函数代码，使你的代码能正常运行。 

**注意：**此版矩阵分解没有 ∑ 矩阵。

In [2]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` 请在 **user_movie_subset** 数据集上尝试你的函数。先将潜在特征设为 4 个，学习速率设为 0.005，并迭代 10 次。在计算生成的 U 和 V 矩阵的点积时，生成的 **user_movie** 矩阵与原始数据子集有何区别？

In [3]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 47.705127
2 		 17.551230
3 		 4.251861
4 		 2.659936
5 		 2.505769
6 		 2.430166
7 		 2.359408
8 		 2.282588
9 		 2.197664
10 		 2.104824


In [ ]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

**请在此处填写你的结论。**

`3.` 再在 **user_movie_subset** 数据集上尝试你的函数。这次潜在特征为 4 个，学习速率设为 0.005。但是，迭代次数提高到 250 次。在计算生成的 U 和 V 矩阵的点积时，生成的 **user_movie** 矩阵与原始数据子集有何区别？迭代 250 次后，误差会怎样？

In [ ]:
user_mat, movie_mat = #use your function with 4 latent features, lr of 0.005 and 250 iterations

In [4]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.10145618  8.68331487 10.85753786 10.00806511]
 [ 8.72566133  7.20005376  9.36152284  8.64490091]
 [ 7.84801193  6.90938616  8.1900834   7.82833147]
 [ 9.29145324  8.15690363 10.61273074  9.29354071]
 [ 8.59370251  6.99066414  9.30557399  8.47409644]
 [ 6.61294921  5.7488069   7.77127849  6.65031736]
 [ 9.14288078  8.34152594  9.49964755  9.20831412]
 [ 8.42717999  6.82160575  8.56712176  8.30672083]
 [ 8.21074637  7.23952162  9.46777587  8.25371352]
 [ 7.89638886  6.62690273  8.12168012  7.54961503]
 [ 8.45981365  7.69426247  9.7296002   8.7760937 ]
 [ 9.69166472  8.66375773  9.75160489  9.62984347]
 [ 9.40799848  8.31031256  9.41443374  9.32511155]
 [ 7.13649075  6.01231265  6.24000655  6.83958295]
 [ 9.817411    8.7128921   9.84775887  9.91238991]
 [ 9.57800602  8.69426939 10.08421886  9.84630933]
 [ 8.4964893   7.65998592  8.10242531  8.44109827]
 [ 7.86888908  6.67299748  5.7843599   7.74813198]
 [ 8.27974295  6.94726366  9.21496812  8.33734109]
 [ 7.70004904  6.84545254  9.32

**请在此处填写你的结论。**

上次在此矩阵里放入 **np.nan** 值时，python 中的整个 SVD 算法崩溃了。我们看看使用你的 FunkSVD 函数是否依然会这样。在以下单元格中，我在你的 numpy 数组的第一个单元格中放入了一个 NaN 值。  

`4.` 假设潜在特征有 4 个，学习速率为 0.005，并且迭代 250 次。你能够运行你的 SVD 并且不崩溃吗（内置的 python 方法会崩溃）？你获得了 NaN 值的预测值吗？缺失值的预测是多少？请在以下单元格中回答这些问题。

In [5]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [6]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 41.769437
2 		 14.106389
3 		 4.192946
4 		 2.947497
5 		 2.762991
6 		 2.698476
7 		 2.659355
8 		 2.625245
9 		 2.590665
10 		 2.553887
11 		 2.514148
12 		 2.470963
13 		 2.423945
14 		 2.372765
15 		 2.317139
16 		 2.256829
17 		 2.191643
18 		 2.121433
19 		 2.046104
20 		 1.965619
21 		 1.880012
22 		 1.789418
23 		 1.694095
24 		 1.594471
25 		 1.491176
26 		 1.385072
27 		 1.277262
28 		 1.169067
29 		 1.061963
30 		 0.957489
31 		 0.857127
32 		 0.762185
33 		 0.673693
34 		 0.592344
35 		 0.518476
36 		 0.452103
37 		 0.392976
38 		 0.340659
39 		 0.294605
40 		 0.254220
41 		 0.218912
42 		 0.188117
43 		 0.161320
44 		 0.138056
45 		 0.117907
46 		 0.100504
47 		 0.085514
48 		 0.072642
49 		 0.061621
50 		 0.052213
51 		 0.044204
52 		 0.037404
53 		 0.031644
54 		 0.026774
55 		 0.022663
56 		 0.019197
57 		 0.016277
58 		 0.013818
59 		 0.011747
60 		 0.010002
61 		 0.008532
62 		 0.007291
63 		 0.006242
64 		

In [7]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 10.440627595271005:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


下面扩展到更真实的例子上。遗憾的是，依然不建议在本地机器上对整个用户-电影矩阵运行此函数。但是，我们能够查看这个示例扩展到 1000 个用户的情况。在上述部分，你使用了一个非常小的数据子集，并且没有缺失值。

`5.` 考虑到此矩阵的大小，运行时间会比较长。假设超参数如下所示：潜在特征为 4 个，学习速率为0.005，迭代次数为20。运行时间会很久，吃点东西，散散步吧。

In [ ]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 23.512568
2 		 10.847862
3 		 7.450251
4 		 5.746853
5 		 4.670920
6 		 3.918359
7 		 3.358859
8 		 2.925480
9 		 2.580222
10 		 2.299717
11 		 2.068517
12 		 1.875795
13 		 1.713645
14 		 1.576140
15 		 1.458750
16 		 1.357947
17 		 1.270918


`6.` 现在你已经获得了每个用户-电影对的预测值，请根据你的结果回答几个问题。请在以下每个变量对应的括号里填写正确的值，并使用以下测试检测你的答案。

In [ ]:
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

In [ ]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

```python

```